In [2]:
# This script uses python 3.12 to load data isolated data and the raw data from each json file into the transaction header table
# Libraries: os, psycopg2,json
# Program uses three steps
#1. Connect to database
#2. Fetch json file based on block number
#3 Pull data from data paths and push to header table
import os
import psycopg2
import json

# Define your database connection parameters
db_params = {
    "host": "localhost",
    "database": "blockchain",
    "user": "postgres",
    "password": "auth2020"
}


block ='1199803' # Input desired block number 


# Input the path to the JSON file
file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'
# Check if files exist
try:
    # Check if the file exists
    if os.path.exists(file_path):
        print(f"Found JSON file: {file_path}")

        # Load JSON data from the file
        with open(file_path, 'r') as json_file:
            json_data = json.load(json_file)

            # Extract other data (type, amount, denom, gas_limit) from your JSON data
            type = json_data["tx"]["body"]["messages"][0]["@type"]
            amount = json_data["tx"]["auth_info"]["fee"]["amount"][0]["amount"]
            denom = json_data["tx"]["auth_info"]["fee"]["amount"][0]["denom"]
            gas_limit = json_data["tx"]["auth_info"]["fee"]["gas_limit"]

            # Convert the entire JSON data to a string
            raw_data = json.dumps(json_data)

            # Connect to the database
            conn = psycopg2.connect(**db_params)
            print("Connected to the database successfully!")

            # Execute the query to retrieve the block_id
            get_block_id_query = f"SELECT block_id FROM block WHERE block = '{block}'"
            with conn.cursor() as cursor:
                cursor.execute(get_block_id_query)
                result = cursor.fetchone()

                if result:
                    block_id = result[0]
                    print(f"Found block_id: {block_id}")

                    # Construct the combined insert query
                    tx_header_insert_query = f"""
                        INSERT INTO transaction_header (block_id, type, amount, denom, gas_limit, raw_data)
                        VALUES ('{block_id}','{type}','{amount}', '{denom}', {gas_limit}, '{raw_data}');
                    """

                    # Execute the insert query
                    cursor.execute(tx_header_insert_query)
                    conn.commit()  # Commit the transaction

                    print("Data inserted successfully!")
                else:
                    print(f"No block_id found for block: {block}")
except Exception as e:
    print(f"Error: {e}")



Found JSON file: C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/1199803.json
Connected to the database successfully!
Found block_id: a6437126-830a-45d5-be47-aac777177778
Data inserted successfully!
